# 大数相乘

大数相乘就是数字特别大，相乘的结果会超出基本数据类型的表示范围，所以这样的数不能直接进行相乘。这种问题可以有三种解决办法：

## Karatsuba算法
karatsuba算法是一种快速相乘算法，采用分治递归的方法求解，此算法在1960年由Anatolii Alexeevitch Karatsuba 提出。普通相乘的复杂度为$O(n^2)$，而Karatsuba算法的复杂度仅为$O(n^{log_2{3}})$。下面来看一下这个算法的实现过程：

假设有两个数 $x = 12345$ 和 $y = 6789$，我们将这两个数分别拆成两部分，即:

$x = a + b$

$y = c + d$


其中 $a = 123 * 10^{n/2}$，$b = 45$，$c = 67 * 10^{n/2}$，$d = 89$，这里 $n = max(len(str(x)), len(str(y)))$
，于是
$$
\begin{aligned} & x * y \\=&\left(a * 10^{n / 2}+b\right) *\left(c * 10^{n / 2}+d\right) \\=& a c * 10^{n}+(a d+b c) * 10^{n / 2}+b d \\=& a c * 10^{n}+[(a+b) *(c+d)-a c-b d] * 10^{n / 2}+b d \end{aligned}
$$
其中，$(a d+b c) = (a+b) *(c+d)-a c-b d$，所以我们可以利用递归，每次只需要每次计算$a c$，$(a+b) *(c+d)$ 和 $b d$，这样四次乘法运算就变成了三次(和六次加减法)。因此复杂度为：
$$T(n)=3 T\left(\frac{n}{2}\right)+6 n=O\left(n^{\log _{2} 3}\right)$$
下面看一个例子，这里指数取为 $n//2$，于是 $a=123$，$b=45$，$c=67$，$a=89$ ：
$$
\begin{aligned} 12345 &=123 \cdot 100+45 \\ 6789 &=67 \cdot 100+89 \end{aligned}
$$
那么 $a*c$，$(a+b) *(c+d)$ 和 $a*c$ 分别为：
$$
\begin{aligned} z_{0} &=a * c=123 \times 67=8241 \\ z_{1} &=b * d=45 \times 89=4005 \\ z_{2} &=((a+b) *(c+d)-a * c-b * d) \\ &=(123+45) \times(67+89)-z_{0}-z_{1}=26208-8241-4005=13962 \end{aligned}
$$
最终结果为：
$$
\begin{array}{l}{\text {ans}=z_{0} \cdot 10^{2 * 2}+z_{2} \cdot 10^{2}+z_{1}} \\ {\text {ans}=8241 \cdot 10^{4}+13962 \cdot 10^{2}+4005=83810205}
\end{array}
$$
至此，我们可以写出整个过程如下：

1. 首先根据两个数的长度进行截取， $n = max(len(str(x)), len(str(y)))$，那么取 $m = n//2$ 作为指数。
2. 计算$a * c$，$(a+b) *(c+d)$ 和 $b * d$
3. 递归计算$a * c$，$(a+b) *(c+d)$ 和 $b * d$
4. 设置递归终止条件

```python
if len(str(x))==1 or len(str(y))==1:
    return x*y
```
下面是具体Python实现。

In [8]:
def karatsubaMul(x, y):
    '''
    Karatsuba算法计算大数相乘
    '''
    if len(str(x)) == 1 or len(str(y)) == 1:
        return x * y
    n = max(len(str(x)), len(str(y)))
    exp = n // 2
    
    a = x // 10**exp
    b = x % 10**exp
    c = y // 10**exp
    d = y % 10**exp
    ac = karatsubaMul(a, c)
    bd = karatsubaMul(b, d)
    abcd = karatsubaMul(a+b, c+d)
    adbc = abcd - ac - bd
    return ac * 10**(2*exp) + adbc * 10**exp + bd

print(karatsubaMul(1234567, 123), 1234567*123)

151851741 151851741



## 模拟乘法手算的累加算法
<div align=center><img src='big_data_mul.png' width=500 /><div/>

模拟手算就像小学的时候计算乘法那样，先拿一个数分别与另一个数的个位、十位、百位、、、依次相乘，然后再错位相加得到计算结果。下面的程序就是按照这个想法实现的。每个变量的含义如下：

$num1:$ 字符串，大数1

$num2:$ 字符串，大数2

$result:$ 列表，存放每个位上的计算结果

$alist:$ 列表，存放num1的每一位整数

$blist:$ 列表，存放num2的每一位整数

$bit_result:$ 存放按位计算的结果

$tens:$ 存放按位计算的结果的进位数


In [19]:
def mulByBit(num1, num2):
    alist = list(map(int, reversed(num1))) # 将字符串反转并转换成整数列表
    blist = list(map(int, reversed(num2)))
    result = [0]*(len(alist) + len(blist))
    for i, vala in enumerate(alist):
        tens = 0
        for j, valb in enumerate(blist):
            bit_result = vala * valb + tens + result[i+j]
            result[i+j] = bit_result % 10
            tens = bit_result // 10
        result[i+j+1] = tens
    result = ''.join(list(map(str, result[::-1])))
    return result if result[0] != '0' else result[1:]

# test
a = '1234567'
b = '123'
print(mulByBit(a, b), int(a)*int(b))

151851741 151851741


## 模拟乘法手算的改进算法

In [28]:
def mulByBit(num1, num2):
    alist = list(map(int, num1))
    blist = list(map(int, num2))
    result = [0]*(len(alist) + len(blist))
    for i in range(len(alist)-1, -1, -1):
        for j in range(len(blist)-1, -1, -1):
            result[i+j] += (alist[i] * blist[j] + result[i+j+1]) // 10
            result[i+j+1] = (alist[i] * blist[j] + result[i+j+1]) % 10
    result = ''.join(map(str, result))
    return  result if result[0] != '0' else result[1:]

# test
a = '123'
b = '456'
print(mulByBit(a, b), int(a)*int(b))

56088 56088


In [38]:
def bitMul(num1, num2):
    alist = list(map(int, num1))
    blist = list(map(int, num2))
    result = [0]*(len(alist) + len(blist))
    for i in range(len(alist)-1, -1, -1):
        for j in range(len(blist)-1, -1, -1):
            result[i+j] += (alist[i] * blist[j] + result[i+j+1]) // 10
            result[i+j+1] = (alist[i] * blist[j] + result[i+j+1]) % 10
    result = ''.join(map(str, result))
    return result if result[0] != '0' else result[1:]

sts = input().strip().split()
print(sts[0], sts[1])
print(bitMul(sts[0], sts[1]))

123 456
123 456
56088


# 最大乘积

给定一个无序数组，包含正数、负数和0，要求从中找出3个数的乘积，使得乘积最大，要求时间复杂度：$O(n)$，空间复杂度：$O(1)$。

输入描述:

无序整数数组A[n]

输出描述:

满足条件的最大乘积

输入例子:

4

3 4 1 2

输出例子:

24

**解决思路:**

考虑到数组中既有正数，又有负数和 $0$ ，且要求出最大乘积，于是分情况讨论如下，其中max1, max2, max2, min1, min2分别为最大的三个数和最小的两个数：
1. 全是正数：则结果应为最大的三个数相乘，即 result = max1 * max2 * max3
2. 全是负数：则结果应为最大的三个负数相乘，即 result = max1 * max2 * max3，和情况1相同
3. 有正有负：这种情况下，结果有两种可能，即 result = min1 * min2 * max1 or result = max1 * max2 * max3

所以，综上可知，result = max(max1 * max2 * max3, min1 * min2 * max1)

In [36]:
def max_three_product(arr):
    min1 = float('inf')
    min2 = float('inf')
    max1 = float('-inf')
    max2 = float('-inf')
    max3 = float('-inf')
    for val in arr:
        if val < min1:
            min1, min2 = val, min1
        elif val < min2:
            min2 = val
        if max1 < val:
            max1, max2, max3 = val, max1, max2
        elif max2 < val:
            max2, max3 = val, max2
        elif max3 < val:
            max3 = val

    return max(max1*max2*max3, min1*min2*max1)

n = input()
arr = list(map(int, input().split()))
print(max_three_product(arr))

6
-1 0 -2 3 -5 7
70
